In [1]:

from openff.toolkit.topology import Molecule
import parmed
from openff.evaluator import unit
from openff.evaluator.substances import Substance, Component, MoleFraction
from openff.evaluator.workflow.schemas import WorkflowSchema

from openff.evaluator.protocols.coordinates import BuildCoordinatesPackmol
from openff.evaluator.protocols.forcefield import BuildSmirnoffSystem
from openff.evaluator.workflow.utils import ProtocolPath

from openff.evaluator.thermodynamics import ThermodynamicState

from openff.evaluator.workflow import Workflow



Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.

****** PyMBAR will use 64-bit JAX! *******
* JAX is currently set to 32-bit bitsize *
* which is its default.                  *
*                                        *
* PyMBAR requires 64-bit mode and WILL   *
* enable JAX's 64-bit mode when called.  *
*                                        *
* This MAY cause problems with other     *
* Uses of JAX in the same code.          *
******************************************



In [2]:

def parameterize_system(smiles, force_field_path="openff_unconstrained-2.1.0.offxml", mass_density=0.95 * unit.grams / unit.milliliters,n_mols=1000):
    # Define the substance
    substance = Substance()
    substance.add_component(Component(smiles=smiles), MoleFraction(1.0))

    # Create the workflow schema
    schema = WorkflowSchema()

    # Step 1: Build coordinates
    build_coordinates = BuildCoordinatesPackmol("build_coordinates")
    build_coordinates.max_molecules = n_mols
    build_coordinates.mass_density = mass_density
    build_coordinates.substance = substance
    schema.protocol_schemas.append(build_coordinates.schema)

    # Step 2: Assign parameters
    assign_parameters = BuildSmirnoffSystem("assign_parameters")
    assign_parameters.force_field_path = force_field_path
    assign_parameters.coordinate_file_path = ProtocolPath("coordinate_file_path", build_coordinates.id)
    assign_parameters.substance = substance
    schema.protocol_schemas.append(assign_parameters.schema)

    # Metadata
    metadata = {
        "substance": substance,
        "thermodynamic_state": ThermodynamicState(
            temperature=298.15 * unit.kelvin,
            pressure=1.0 * unit.atmosphere
        ),
        "force_field_path": force_field_path
    }

    # Create and execute the workflow
    workflow = Workflow.from_schema(schema, metadata=metadata)
    workflow.execute()

    return workflow


In [18]:
import mdtraj as md
import openmm
from openmm.app.pdbfile import PDBFile
import parmed
def parameterize_system(smiles, force_field_path="openff_unconstrained-2.1.0.offxml", mass_density=0.95 * unit.grams / unit.milliliters,n_mols=1000):
    # Define the substance
    substance = Substance()
    substance.add_component(Component(smiles=smiles), MoleFraction(1.0))

    # Create the workflow schema
    schema = WorkflowSchema()

    # Step 1: Build coordinates
    build_coordinates = BuildCoordinatesPackmol("build_coordinates")
    build_coordinates.max_molecules = n_mols
    build_coordinates.mass_density = mass_density
    build_coordinates.substance = substance
    schema.protocol_schemas.append(build_coordinates.schema)

    # Step 2: Assign parameters
    assign_parameters = BuildSmirnoffSystem("assign_parameters")
    assign_parameters.force_field_path = force_field_path
    assign_parameters.coordinate_file_path = ProtocolPath("coordinate_file_path", build_coordinates.id)
    assign_parameters.substance = substance
    schema.protocol_schemas.append(assign_parameters.schema)

    # Metadata
    metadata = {
        "substance": substance,
        "thermodynamic_state": ThermodynamicState(
            temperature=298.15 * unit.kelvin,
            pressure=1.0 * unit.atmosphere
        ),
        "force_field_path": force_field_path
    }

    # Create and execute the workflow
    workflow = Workflow.from_schema(schema, metadata=metadata)
    workflow.execute()

    pdb_file = f'{workflow.schema.id}_build_coordinates/output.pdb'
    parameterized_system = f'{workflow.schema.id}_assign_parameters/system.xml'
    omm_top = PDBFile(pdb_file).topology
    parmed_obj = parmed.openmm.load_topology(omm_top, parameterized_system)

    return parmed_obj

In [20]:
from openff.toolkit.typing.engines.smirnoff import ForceField
ff = ForceField("openff_unconstrained-2.1.0.offxml")
#get path to the force field file


In [4]:
# Example usage
smiles = "c1ccccc1"  # Benzene SMILES
workflow = parameterize_system(smiles)
# pdb_file = f'/localhome/cschiebroek/MDFP_VP/mdfptools/carl/{workflow.schema.id}_build_coordinates/output.pdb'
# parameterized_system = f'/localhome/cschiebroek/MDFP_VP/mdfptools/carl/{workflow.schema.id}_assign_parameters/system.xml'
# from openmm.app.pdbfile import PDBFile
# omm_top = PDBFile(pdb_file).topology
# parmed_obj = parmed.openmm.load_topology(omm_top, parameterized_system)
# print(workflow.schema.id)

In [1]:
import Parameteriser  
smiles = "c1ccccc1"  # Benzene SMILES
rdk_pmd = Parameteriser.LiquidParameteriser.via_rdkit(smiles=smiles)

Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.

****** PyMBAR will use 64-bit JAX! *******
* JAX is currently set to 32-bit bitsize *
* which is its default.                  *
*                                        *
* PyMBAR requires 64-bit mode and WILL   *
* enable JAX's 64-bit mode when called.  *
*                                        *
* This MAY cause problems with other     *
* Uses of JAX in the same code.          *
******************************************



In [2]:
rdk_pmd

<Structure 1200 atoms; 100 residues; 1200 bonds; PBC (orthogonal); parameterized>

In [3]:
#print penff.evaluator version
import openff.evaluator
openff.evaluator.__version__

'0.1.0-beta.4+226.gca084df.dirty'

In [4]:
#and openff.toolkit version
import openff.toolkit
openff.toolkit.__version__

'0.16.1'

In [4]:
rdk_pmd = Parameteriser.SolutionParameteriser.via_rdkit(smiles=smiles)


/localhome/cschiebroek/.conda/envs/openff-evaluator/lib/python3.11/site-packages/parmed/structure.py:1799: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  coords = np.array(value, dtype=np.float64, copy=False, subok=True)


In [19]:
smiles = "c1ccccc1"  # Benzene SMILES
prm_obj = parameterize_system(smiles)
prm_obj

<Structure 12000 atoms; 1000 residues; 12000 bonds; PBC (orthogonal); parameterized>

In [11]:
workflow.schema.protocol_schemas[0].get_attributes()

['id', 'type', 'inputs']

In [14]:
workflow.schema.protocol_schemas[0].id
#get path to pdb file
path_to_pdb = f'{workflow.schema.id}_build_coordinates/output.pdb'
path_to_prmd_system = f'{workflow.schema.id}_assign_parameters/system.xml'

TypeError: list indices must be integers or slices, not str

In [ ]:
#get path to pdb file
workflow.schema

In [32]:
# Example usage
smiles = "c1ccccc1"  # Benzene SMILES
workflow = parameterize_system(smiles,n_mols=100)
pdb_file = f'/localhome/cschiebroek/MDFP_VP/mdfptools/carl/{workflow.schema.id}_build_coordinates/output.pdb'
parameterized_system = f'/localhome/cschiebroek/MDFP_VP/mdfptools/carl/{workflow.schema.id}_assign_parameters/system.xml'
from openmm.app.pdbfile import PDBFile
omm_top = PDBFile(pdb_file).topology
parmed_obj = parmed.openmm.load_topology(omm_top, parameterized_system)
print(workflow.schema.id)

adb70b0f9c8b43a9a9373ada67a25377


In [33]:
# Example usage
smiles = "c1ccccc1"  # Benzene SMILES
workflow = parameterize_system(smiles,n_mols=256)
pdb_file = f'/localhome/cschiebroek/MDFP_VP/mdfptools/carl/{workflow.schema.id}_build_coordinates/output.pdb'
parameterized_system = f'/localhome/cschiebroek/MDFP_VP/mdfptools/carl/{workflow.schema.id}_assign_parameters/system.xml'
from openmm.app.pdbfile import PDBFile
omm_top = PDBFile(pdb_file).topology
parmed_obj = parmed.openmm.load_topology(omm_top, parameterized_system)
print(workflow.schema.id)

f09a080a01674150b8116e28bfe506dc


In [12]:
import pandas as pd
df = pd.read_csv('/localhome/cschiebroek/MDFP_VP/mdfptools/carl/filtered_frame_densities_ThermoMLData.csv')

In [17]:
#filter df_organic, with only organic molecules (At least one carbon atom)
df_organic = df[df['Component 1'].str.contains('C')]
df_organic = df[df['Component 1'].str.contains('C')]

In [20]:
#lets look at two extremes, to see if values converge. Very low:
df_organic.sort_values('Density Value (g / ml)').head(1)

,Unnamed: 0,Id,Temperature (K),Pressure (kPa),Phase,N Components,Component 1,Role 1,Mole Fraction 1,Density Value (g / ml),Density Uncertainty (g / ml),Source
53666,453721,0ea18c37dc8340b289b96d43e59c0fba,308.15,101.325,Liquid,1,CCCCC,Solvent,1.0,0.61101,0.000385,10.1016/j.jct.2005.05.020


In [22]:
df.sort_values('Density Value (g / ml)',ascending=False).head(2)

,Unnamed: 0,Id,Temperature (K),Pressure (kPa),Phase,N Components,Component 1,Role 1,Mole Fraction 1,Density Value (g / ml),Density Uncertainty (g / ml),Source
44419,369346,3f010e87a0084c0fb67a672f7f65e609,298.15,101.325,Liquid,1,[Hg],Solvent,1.0,13.5300,0.015000,10.1016/j.jct.2018.12.016
64946,549179,921ee6d5505449628f28d314fb9ff7c3,293.15,101.000,Liquid,1,BrC(Br)C(Br)Br,Solvent,1.0,2.9632,0.001765,10.1016/j.jct.2005.03.027


In [26]:
for smiles in  ['CCCCC','BrC(Br)C(Br)Br']:
    workflow = parameterize_system(smiles)
    print(smiles,workflow.schema.id)

CCCCC 899401215785451f8733675e24b98a18
BrC(Br)C(Br)Br e1baf18c2527499f8ce274620e805a09


In [27]:
workflow = parameterize_system('CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC')
print(smiles,workflow.schema.id)

KeyboardInterrupt: 